In [1]:
from dataprocess.kdd_cup99 import KDD_CUP_99_DataLoader
from net.CNN_KDD import CNN_KDD
from net.multCNN import multCNN

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [14]:
import os 
import time

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

t = time.localtime()
year, month, day = t.tm_year, t.tm_mon, t.tm_mday

mode = input('input the tag about the training: ')
if mode == '':
    mode = 'no_tag'
log_directory = f"/Transfer-learning-IDS/log/exp/multCNN/{mode}/{month}_{day}/"
ckpt_directory = f"/Transfer-learning-IDS/history/multCNN/{mode}/checkpoint/{month}_{day}/"
if not os.path.isdir(log_directory):
    os.makedirs(log_directory)
if not os.path.isdir(ckpt_directory):
    os.makedirs(ckpt_directory)
print('log_directory: ', log_directory)
print('ckpt_directory: ', ckpt_directory)


cuda
log_directory:  /Transfer-learning-IDS/log/exp/multCNN/kddcup_balance/4_21/
ckpt_directory:  /Transfer-learning-IDS/history/multCNN/kddcup_balance/checkpoint/4_21/


In [4]:
dataset = KDD_CUP_99_DataLoader('E:/DataSets/kddcup.data', 256)
discrete_column_idx = dataset.data.discrete_column
discrete_column_idx.remove(41)
continuous_column_idx = [i for i in range(41) if i not in discrete_column_idx]
input_channel_kind = len(discrete_column_idx)
input_channel_num = len(continuous_column_idx)

In [5]:
# net = CNN_KDD(41,23,3).to(device)
net = multCNN(input_channel_num, input_channel_kind, 23).to(device)
epoch = 100
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=1e-1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25)  

In [12]:
writer1 = SummaryWriter(log_directory)

<class 'str'>


In [9]:
# ckpt = torch.load('/Transfer-learning-IDS/history/CNN_KDD/checkpoint/ckpt_best_99.pth')
# epoch_start = ckpt["epoch"]
# net.load_state_dict(ckpt["net"])
# optimizer.load_state_dict(ckpt["optimizer"])

In [7]:
for t in range(0, epoch):
    running_loss = 0
    for step, (x, y) in enumerate(dataset):
        
        x_continuous, x_discrete =x[:, continuous_column_idx], x[:, discrete_column_idx]
        x_continuous = x_continuous.to(device)
        x_discrete = x_discrete.to(device)
        y = F.one_hot(y.long(), num_classes=23).float()
        y = y.to(device)
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        y_pred = net((x_continuous, x_discrete))
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        # visualize loss
        running_loss += loss.item()
    # ...log the running loss
    writer1.add_scalar('training loss', running_loss, t)
    print("epoch: %d, learning rate: %f" % (t, optimizer.param_groups[0]['lr']))
    scheduler.step()
    if t % 10 == 9:
        checkpoint = {"net": net.state_dict(), 'optimizer':optimizer.state_dict(), "epoch": t}
        torch.save(checkpoint,  ckpt_directory + 'ckpt_best_%s.pth' %(str(t)))
writer1.close()

epoch: 0, learning rate: 0.100000
epoch: 1, learning rate: 0.100000
epoch: 2, learning rate: 0.100000
epoch: 3, learning rate: 0.100000
epoch: 4, learning rate: 0.100000
epoch: 5, learning rate: 0.100000
epoch: 6, learning rate: 0.100000
epoch: 7, learning rate: 0.100000
epoch: 8, learning rate: 0.100000
epoch: 9, learning rate: 0.100000
epoch: 10, learning rate: 0.100000
epoch: 11, learning rate: 0.100000
epoch: 12, learning rate: 0.100000
epoch: 13, learning rate: 0.100000
epoch: 14, learning rate: 0.100000
epoch: 15, learning rate: 0.100000
epoch: 16, learning rate: 0.100000
epoch: 17, learning rate: 0.100000
epoch: 18, learning rate: 0.100000
epoch: 19, learning rate: 0.100000
epoch: 20, learning rate: 0.025000
epoch: 21, learning rate: 0.025000
epoch: 22, learning rate: 0.025000
epoch: 23, learning rate: 0.025000
epoch: 24, learning rate: 0.025000
epoch: 25, learning rate: 0.025000
epoch: 26, learning rate: 0.025000
epoch: 27, learning rate: 0.025000
epoch: 28, learning rate: 0.02